<a href="https://colab.research.google.com/github/mbyanfei/Google-Colab/blob/master/AB_NLP_seminar_20220517.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

Using custom data configuration default
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/ccdv___cnn_dailymail/default/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)


  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


In [8]:
for i in range(0, 2000):
    if 'stock market' in dataset["train"][i]["article"]:
        print(i)

395
1148


In [9]:
sample = dataset["train"][1148]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:2000])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4242):

Paris, France (CNN)  -- The future of social networking, the real-time Web and a host of apps and gadgets were the talk of the annual Le Web conference as it opened here Wednesday. The phenomenal success of microblogging site Twitter and its plans for the future dominated the two-day event's opening sessions for bloggers and industry leaders, held in a converted morgue in the city's northeastern suburbs. Top executives from Twitter faced off with rivals from Facebook and MySpace as all three social media sites unveiled technical innovations designed to capture more users. On the heels of Twitter founder Jack Dorsey's unveiling of Square, a device that accepts credit card payments on cell phones, Twitter platform director Ryan Sarver said the site now has 50,000 dedicated applications making use of its constant stream of blog updates. Sarver also announced the company's "firehose" feature, which throws open its data stream to an

In [10]:
sample_text = dataset["train"][1148]["article"][:2000]
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

In [13]:
import nltk
from nltk.tokenize import sent_tokenize

In [14]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [11]:
sample_text

'Paris, France (CNN)  -- The future of social networking, the real-time Web and a host of apps and gadgets were the talk of the annual Le Web conference as it opened here Wednesday. The phenomenal success of microblogging site Twitter and its plans for the future dominated the two-day event\'s opening sessions for bloggers and industry leaders, held in a converted morgue in the city\'s northeastern suburbs. Top executives from Twitter faced off with rivals from Facebook and MySpace as all three social media sites unveiled technical innovations designed to capture more users. On the heels of Twitter founder Jack Dorsey\'s unveiling of Square, a device that accepts credit card payments on cell phones, Twitter platform director Ryan Sarver said the site now has 50,000 dedicated applications making use of its constant stream of blog updates. Sarver also announced the company\'s "firehose" feature, which throws open its data stream to any developers who want to use it. This is a first for T

In [16]:
summaries["baseline"] = three_sentence_summary(sample_text)

In [17]:
from transformers import pipeline, set_seed

In [ ]:
set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

In [18]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [19]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

**PEGASUS**

In [20]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

ValueError: ignored

In [21]:
print("GROUND TRUTH")
print(dataset["train"][1148]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
The future of social networking was the talk of the Le Web conference as it opened in Paris .
Twitter, Facebook and MySpace all unveiled innovations designed to capture more users .
Twitter's Ryan Sarver: Site now has 50,000 dedicated applications .
YouTube's Chad Hurley: There's no secret formula for creating a viral video sensation .

BASELINE
Paris, France (CNN)  -- The future of social networking, the real-time Web and a host of apps and gadgets were the talk of the annual Le Web conference as it opened here Wednesday.
The phenomenal success of microblogging site Twitter and its plans for the future dominated the two-day event's opening sessions for bloggers and industry leaders, held in a converted morgue in the city's northeastern suburbs.
Top executives from Twitter faced off with rivals from Facebook and MySpace as all three social media sites unveiled technical innovations designed to capture more users.

T5
the future of social networking, the real-time Web and a

In [22]:
from datasets import load_metric
import pandas as pd
import numpy as np

In [ ]:
!pip install sacrebleu

In [25]:
bleu_metric = load_metric("sacrebleu") 

In [26]:
BLUE_results = pd.DataFrame()
for model_name in summaries:
    bleu_metric.add(
        prediction=summaries[model_name], reference=[dataset["train"][1148]["highlights"]])
    results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
    results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
    BLUE_results = pd.concat([BLUE_results, pd.DataFrame.from_dict(results, orient="index", columns=[model_name])], axis=1)

BLUE_results

,baseline,t5,bart
score,18.092628,19.402323,13.865641
counts,"[35, 20, 15, 10]","[27, 15, 11, 7]","[26, 11, 7, 3]"
totals,"[101, 100, 99, 98]","[70, 69, 68, 67]","[65, 64, 63, 62]"
precisions,"[34.65, 20.0, 15.15, 10.2]","[38.57, 21.74, 16.18, 10.45]","[40.0, 17.19, 11.11, 4.84]"
bp,1.0,1.0,1.0
sys_len,101,70,65
ref_len,59,59,59


In [ ]:
pip install rouge_score

In [29]:
rouge_metric = load_metric("rouge")

In [30]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.096000,0.0,0.080000,0.096000
t5,0.062500,0.0,0.041667,0.062500
bart,0.065934,0.0,0.043956,0.065934


In [1]:
!pip freeze | grep spacy

en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.5/en_core_web_sm-2.2.5.tar.gz
spacy==3.3.0
spacy-legacy==3.0.9
spacy-loggers==1.0.2


In [ ]:
pip install -U spacy

In [ ]:
!pip install tf-estimator-nightly==2.8.0.dev2021122109

In [ ]:
!pip install spacy[transformers]

In [9]:
import spacy
import en_core_web_trf

In [4]:
spacy_model_name = 'en_core_web_trf'
if not spacy.util.is_package(spacy_model_name):
    spacy.cli.download(spacy_model_name)
nlp = spacy.load(spacy_model_name)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [6]:
text = "Biden requests $33 billion for Ukraine war; Putin threatens ‘lightning fast’ retaliation to nations that intervene."

In [7]:
doc = nlp(text)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [8]:
for entity in doc.ents: 
    print(entity.text, entity.label_)

Biden PERSON
$33 billion MONEY
Ukraine GPE
Putin PERSON


In [10]:
for label in nlp.get_pipe("ner").labels: 
    print(label+': ', spacy.explain(label))

CARDINAL:  Numerals that do not fall under another type
DATE:  Absolute or relative dates or periods
EVENT:  Named hurricanes, battles, wars, sports events, etc.
FAC:  Buildings, airports, highways, bridges, etc.
GPE:  Countries, cities, states
LANGUAGE:  Any named language
LAW:  Named documents made into laws.
LOC:  Non-GPE locations, mountain ranges, bodies of water
MONEY:  Monetary values, including unit
NORP:  Nationalities or religious or political groups
ORDINAL:  "first", "second", etc.
ORG:  Companies, agencies, institutions, etc.
PERCENT:  Percentage, including "%"
PERSON:  People, including fictional
PRODUCT:  Objects, vehicles, foods, etc. (not services)
QUANTITY:  Measurements, as of weight or distance
TIME:  Times smaller than a day
WORK_OF_ART:  Titles of books, songs, etc.


In [11]:
from transformers import pipeline

In [12]:
ner_pipe = pipeline("ner")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

In [13]:
for entity in ner_pipe(text):
    print(entity)

{'entity': 'I-PER', 'score': 0.97980887, 'index': 1, 'word': 'B', 'start': 0, 'end': 1}
{'entity': 'I-PER', 'score': 0.98380405, 'index': 2, 'word': '##iden', 'start': 1, 'end': 5}
{'entity': 'I-LOC', 'score': 0.99976796, 'index': 8, 'word': 'Ukraine', 'start': 31, 'end': 38}
{'entity': 'I-PER', 'score': 0.998776, 'index': 11, 'word': 'Putin', 'start': 44, 'end': 49}
